In [1]:
import pandas as pd
import regex as re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.tokenize import word_tokenize
import nltk
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('punkt')
pd.set_option('display.max_rows',None)
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Data MUA for Memory Based System Recommendation

In [2]:
class ContentBasedFiltering:
    def case_folding(self, text):
        if type(text) == str:
            text = text.lower()
        return text

    def punctuation_removal(self, text):
        if type(text) == str:
            text = re.sub(r'[^\w\s]', '', text)
        return text

    def tokenize(self, text):
        tokens = word_tokenize(text)
        return tokens
        
    def stopwords_removal(self, text):
        stopword_factory = StopWordRemoverFactory()
        stopword_remover = stopword_factory.create_stop_word_remover()
        no_stopword = stopword_remover.remove(text)
        return no_stopword

    def preprocessing(self, text):
        text = self.case_folding(text)
        text = self.punctuation_removal(text)
        text = self.stopwords_removal(text)
        return text

In [3]:
data = pd.read_excel('./web_mua/static/Data MUA 2.xlsx')
data.head()

,no,nama,nama_MUA,latitude,longitude,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating
0,1,Anggia Pradita A,Intan Rias Pengantin,-8.233613,113.686384,la tulipe cover foundation,natural (putih),light,cool,Murah,-8.233613,113.66733,4
1,1,Anggia Pradita A,Intan Rias Pengantin,-8.233613,113.686384,la tulipe cover foundation,suntan (sawo matang),medium,warm,Murah,-8.233613,113.66733,4
2,2,VIKA APRILIA SINTA PUTRI,Sanggar Rias Cantik Tina,-8.221109,113.680964,la tulipe cover foundation,rachel 1 (kuning langsat),medium,neutral,Murah,-8.233613,113.66733,5
3,2,VIKA APRILIA SINTA PUTRI,Sanggar Rias Cantik Tina,-8.221109,113.680964,la tulipe cover foundation,suntan (sawo matang),medium,warm,Murah,-8.233613,113.66733,5
4,3,septia maharani,Rias Pengantin Azizah,-8.234863,113.680670,la tulipe cover foundation,natural (putih),light,cool,Sedang,-8.233613,113.66733,3


In [4]:
def cleaning_data(data):
    cleaned_data = []
    CBF = ContentBasedFiltering()
    for i in range(len(data)):
        cleaned_data.append(CBF.preprocessing(str(data[i])))
    return cleaned_data

In [5]:
#praproses data untuk nama
data['nama'] = cleaning_data(data['nama'])

In [6]:
data_for_model = data.copy()
data_for_user = data.copy()

#praproses data untuk deskripsi
data_for_model['desc'] = data['produk_makeup'] + ' ' + data['shade'] + ' ' + data['skin_color'] + ' ' + data['skin_undertone']
data_for_model['desc'] = cleaning_data(data_for_model['desc'])
data_for_user['desc'] = data['produk_makeup'] + ' ' + data['skin_color'] + ' ' + data['skin_undertone']
data_for_user['desc'] = cleaning_data(data_for_user['desc'])

In [7]:
# data_for_model = data_for_model.groupby('no').agg({'nama':'first', 'nama_MUA':'first', 'kategori_harga': 'first', 'latitude':'first', 'longtitude':'first', 'rating':'first','desc':' '.join}).reset_index()
data_for_model = data_for_model.sort_values(by=['nama'], ignore_index=True)

data_for_user = data_for_user.sort_values(by=['nama'], ignore_index=True)
data_for_user = data_for_user.drop(columns=['shade'])

In [8]:
data_for_model.head()

,no,nama,nama_MUA,latitude,longitude,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating,desc
0,53,adera zia,Dinda Wijaya Make Up,-8.193711,113.619405,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,Sedang,-8.193711,113.616830,5,lt pro smooth corrector cream foundation natur...
1,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,Sedang,-8.139019,113.728548,4,lt pro smooth corrector cream foundation natur...
2,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,la tulipe cover foundation,suntan (sawo matang),medium,warm,Sedang,-8.139019,113.728548,4,la tulipe cover foundation suntan sawo matang ...
3,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,la tulipe cover foundation,suntan (sawo matang),medium,warm,Sedang,-8.157255,113.652074,3,la tulipe cover foundation suntan sawo matang ...
4,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,LT Pro smooth corrector cream foundation,plum rose (putih),light,cool,Sedang,-8.157255,113.652074,3,lt pro smooth corrector cream foundation plum ...


In [9]:
data_for_user.head()

,no,nama,nama_MUA,latitude,longitude,produk_makeup,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating,desc
0,53,adera zia,Dinda Wijaya Make Up,-8.193711,113.619405,LT Pro smooth corrector cream foundation,medium,neutral,Sedang,-8.193711,113.616830,5,lt pro smooth corrector cream foundation mediu...
1,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,LT Pro smooth corrector cream foundation,medium,neutral,Sedang,-8.139019,113.728548,4,lt pro smooth corrector cream foundation mediu...
2,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,la tulipe cover foundation,medium,warm,Sedang,-8.139019,113.728548,4,la tulipe cover foundation medium warm
3,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,la tulipe cover foundation,medium,warm,Sedang,-8.157255,113.652074,3,la tulipe cover foundation medium warm
4,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,LT Pro smooth corrector cream foundation,light,cool,Sedang,-8.157255,113.652074,3,lt pro smooth corrector cream foundation light...


In [10]:
data_for_model.shape

(179, 14)

In [11]:
from sklearn.preprocessing import LabelEncoder

harga_labels = LabelEncoder()
harga_labels = harga_labels.fit(data['kategori_harga'])
harga_labels.classes_ = np.array(['Murah', 'Sedang', 'Mahal'])
data_for_model['kategori_harga'] = harga_labels.transform(data_for_model['kategori_harga'])
data_for_user['kategori_harga'] = harga_labels.transform(data_for_user['kategori_harga'])

In [12]:
data_for_model.head()

,no,nama,nama_MUA,latitude,longitude,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating,desc
0,53,adera zia,Dinda Wijaya Make Up,-8.193711,113.619405,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,1,-8.193711,113.616830,5,lt pro smooth corrector cream foundation natur...
1,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,1,-8.139019,113.728548,4,lt pro smooth corrector cream foundation natur...
2,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,la tulipe cover foundation,suntan (sawo matang),medium,warm,1,-8.139019,113.728548,4,la tulipe cover foundation suntan sawo matang ...
3,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,la tulipe cover foundation,suntan (sawo matang),medium,warm,1,-8.157255,113.652074,3,la tulipe cover foundation suntan sawo matang ...
4,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,LT Pro smooth corrector cream foundation,plum rose (putih),light,cool,1,-8.157255,113.652074,3,lt pro smooth corrector cream foundation plum ...


In [13]:
data_for_user.head()

,no,nama,nama_MUA,latitude,longitude,produk_makeup,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating,desc
0,53,adera zia,Dinda Wijaya Make Up,-8.193711,113.619405,LT Pro smooth corrector cream foundation,medium,neutral,1,-8.193711,113.616830,5,lt pro smooth corrector cream foundation mediu...
1,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,LT Pro smooth corrector cream foundation,medium,neutral,1,-8.139019,113.728548,4,lt pro smooth corrector cream foundation mediu...
2,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,la tulipe cover foundation,medium,warm,1,-8.139019,113.728548,4,la tulipe cover foundation medium warm
3,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,la tulipe cover foundation,medium,warm,1,-8.157255,113.652074,3,la tulipe cover foundation medium warm
4,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,LT Pro smooth corrector cream foundation,light,cool,1,-8.157255,113.652074,3,lt pro smooth corrector cream foundation light...


## User Input

In [14]:
# use this parameter to get some MUA recommendations
def data_preparation(data_for_model, data_for_user, index_user, user):
    du = data_for_user.copy()
    query_data = du.iloc[[index_user]].reset_index()
    query_data_todict = query_data.to_dict('index')[0]
    part_of_model = data_for_model.copy()
    part_of_model = part_of_model.loc[part_of_model.nama_MUA != query_data.nama_MUA[0]].reset_index()
    part_of_query = part_of_model.loc[part_of_model['nama'] == user].reset_index()
    part_of_model = part_of_model.reset_index()
    part_of_model = part_of_model.drop(columns='index')
    return query_data_todict, part_of_query, part_of_model

# example of input data
find_user = data_for_user.index[data_for_user['nama'] == 'adera zia'].to_list()
txt_user = 'adera zia'
query_data_todict, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, find_user[0], txt_user)
# part_of_model = data_preparation(data_for_model, data_for_user, find_user[0], txt_user, first=False)

## Calculating Cosine Similarity for Content Based Model

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity

def tf_idf_vector(part_of_model, query_data_todict):
    copy_md = part_of_model.copy()
    md = pd.DataFrame(copy_md["desc"])
    md.loc[len(md)] = query_data_todict["desc"]
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(md["desc"])
    model = X[:len(part_of_model)]
    query = X[len(part_of_model):]
    return model, query

In [16]:
from geopy.distance import great_circle
import math
def calculate_distance(query_data_todict, part_of_model):
        loc1 = (query_data_todict['latitude'], query_data_todict['longtitude'])
        distance_loc = []
        for i in range(len(part_of_model.index)):
            loc2 = (part_of_model['latitude'][i], part_of_model['longtitude'][i])
            tmp = great_circle(loc1, loc2).km
            value = math.ceil(tmp * 100) / 100
            distance_loc.append(value)
        return distance_loc

In [17]:
def get_numtext_content(query_data_todict, part_of_model):
    tfidf_model, tfidf_query = tf_idf_vector(part_of_model, query_data_todict)
    cos_sim = cosine_similarity(tfidf_query, tfidf_model)
    data_sim = cos_sim.reshape(-1,1)
    distance = calculate_distance(query_data_todict, part_of_model)
    return data_sim, distance

In [18]:
from sklearn.preprocessing import StandardScaler

def get_values(part_of_model):
    tmp_data = part_of_model[['kategori_harga','sim','distance']].to_numpy()
    sim_num = cosine_similarity(tmp_data)
    sim_num = np.array([np.mean(p) for p in sim_num])
    return sim_num

In [19]:
def value_of_mua(query_data_todict, part_of_model):
    sim, distance = get_numtext_content(query_data_todict, part_of_model)
    part_of_model["sim"] = sim
    part_of_model["distance"] = distance
    sf = []
    for i in range(len(part_of_model)):
        sf.append(part_of_model["sim"][i]/(part_of_model["distance"][i] + 1))
    part_of_model["sim_final"] = sf   
    sim_num = get_values(part_of_model)
    part_of_model["sim_num"] = sim_num
    part_of_model = part_of_model.sort_values(by=['nama_MUA'])
    nama_mua = part_of_model["nama_MUA"].unique()
    value_mua = np.empty(len(nama_mua), dtype=float)
    for i in range(len(nama_mua)):
        tmp_val = []
        for j in range(len(part_of_model.index)):
            if nama_mua[i] == part_of_model.nama_MUA[j]:
                tmp_val.append(part_of_model["sim_num"][j] + part_of_model["sim_final"][j])
        value_mua[i] = np.mean(tmp_val)
    part_of_model['total_mua'] = part_of_model["sim_num"] + part_of_model["sim_final"]
    part_of_model = part_of_model.reset_index()
    part_of_model = part_of_model.drop(columns='index')
    return nama_mua, value_mua, part_of_model

## FIND USER

In [20]:
unique_user = data_for_model['nama'].unique()
unique_user
query_user = ['adera zia', 'anggia pradita a', 'anissa serina', 'calista devi aqilah', 'callista febriana erwanto', 'dwi rahma', 'maulina vira', 
              'vika aprilia sinta putri', 'yuricha mahargiyani wijaya', 'zainata hanum masruroh', 'zyanti laila']
selected_user = []
for i in query_user:
    find_user = data_for_model.index[data_for_model['nama'] == i].to_list()
    selected_user.append(find_user[0])
selected_user

[0, 10, 20, 33, 45, 73, 101, 142, 151, 161, 169]

## HYBRID FILTERING

In [21]:
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import preprocessing as pre
import statistics 

In [22]:
#chosen user for testing
def hybrid_evaluation(data_for_model, data_for_user, selected_user, query_user):
    mae_hy = []
    for u in range(len(selected_user)):
        query_data_todict, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[u], query_user[u])
        nama_mua, value_mua, part_of_model = value_of_mua(query_data_todict, part_of_model)
        pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        col_model = pivot.columns.tolist()
        val_model = np.array(pivot.values.tolist(), dtype=float)
        nama_user = np.array(pivot.index.tolist())
        pivot_query = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_query = pivot_query.values.tolist()
        col_query = pivot_query.columns.tolist()
        feature = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_query)):
                if col_model[i] == col_query[j]:
                    feature[i] = val_query[0][j]
                    break
        cos_sim_user = cosine_similarity([feature], val_model)

        weight_user = np.empty(len(part_of_model), dtype=float)
        for i in range(len(part_of_model)):
            for j in range(len(nama_user)):
                if part_of_model["nama"][i] == nama_user[j]:
                    weight_user[i] = cos_sim_user[0][j]
        
        weight_content = np.empty(len(part_of_model), dtype=float)
        for i in range(len(part_of_model)):
            for j in range(len(nama_mua)):
                if part_of_model["nama_MUA"][i] == nama_mua[j]:
                    weight_content[i] = value_mua[j]
        
        final_res = part_of_model.copy()
        final_res = final_res[['nama', 'nama_MUA']]
        final_res['user'] = weight_user
        final_res['content'] = weight_content
        
        mean_weight = np.zeros(len(part_of_model), dtype=float)
        for i in range(len(part_of_model)):
            mean_weight[i] = statistics.harmonic_mean([weight_user[i] + np.min(weight_user[np.nonzero(weight_user)]),  weight_content[i]]) 
        final_res['mean_'] = mean_weight
        higher_value = final_res.copy()
        higher_value = final_res.groupby("nama", as_index=False).mean_.agg(["mean"])
        u = final_res.groupby("nama", as_index=False).user.agg(["mean"])
        higher_value['weight_user'] = u['mean']
        c = final_res.groupby("nama", as_index=False).content.agg(["mean"])
        higher_value['weight_content'] = c['mean']
        higher_value = higher_value.sort_values(by=['mean'], ignore_index=True, ascending=False)
        list_nama = higher_value.nama[:10].tolist()
        pivot = part_of_model.copy()
        pivot['mean_'] = mean_weight
        pivot = pivot[pivot.nama.isin(list_nama)].reset_index()
        pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
        pivot_['weight'] = pivot.groupby("nama_MUA").mean_.agg(['mean'])
        value_rating = np.zeros(len(col_model), dtype=float)
        for i in range(len(pivot_.index)):
            for j in range(len(col_model)):
                if value_rating[j] == 0.0:
                    if col_model[j] == pivot_.index[i]:
                        if pivot_['weight'][i] != 0.0:
                            value_rating[j] = pivot_["mean"][i] * pivot_['weight'][i]
                        else:
                            value_rating[j] = weight_content[i]
                else:
                    continue
        
        tmp_data_recs = {'nama_mua':nama_mua, 'score':value_rating}
        mua_recs = pd.DataFrame(tmp_data_recs).sort_values(by=['score'], ignore_index=True, ascending=False)
        pivot_label = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_label = pivot_label.values.tolist()
        col_label = pivot_label.columns.tolist()
        final_label = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_label)):
                if col_model[i] == col_label[j]:
                    final_label[i] = val_label[0][j]
                    break
        mae = mean_absolute_error(value_rating, final_label)
        mae_hy.append(mae)
        
    mean_mae_hy = np.mean(mae_hy)
    return mua_recs, mean_mae_hy, mae_hy

In [23]:
hybrid_based_fil, mae_hyb, mae_eval = hybrid_evaluation(data_for_model, data_for_user, selected_user, query_user)

In [24]:
hybrid_based_fil[hybrid_based_fil.score > 0.0]

,nama_mua,score
0,Ayuk Rias Pengantin,4.360696
1,Larasnada Make Up,4.201493
2,Ayunia Bridal dan Salon,4.094906
3,Laura Make Up & Bridal,4.092151
4,Ulfa Beauty MUA,3.494019
5,Dinda Wijaya Make Up,3.493168
6,Liyan Make Up Jember,3.479377
7,Bonita MUA Jember,3.353178
8,Eka Make Over,3.233374
9,Griya Putri Make Up,2.824658


In [25]:
mae_hyb

0.735605826082887

## TESTING

In [26]:
def testing_user(data_for_model, data_for_user, index_usr):
    query_data_todict, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[index_usr], query_user[index_usr])
    nama_mua, value_mua, part_of_model = value_of_mua(query_data_todict, part_of_model)
    pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
    col_model = pivot.columns.tolist()
    val_model = np.array(pivot.values.tolist(), dtype=float)
    nama_user = np.array(pivot.index.tolist())
    pivot_query = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
    val_query = pivot_query.values.tolist()
    col_query = pivot_query.columns.tolist()
    feature = np.zeros(len(col_model), dtype=float)
    for i in range(len(col_model)):
        for j in range(len(col_query)):
            if col_model[i] == col_query[j]:
                feature[i] = val_query[0][j]
                break
    cos_sim_user = cosine_similarity([feature], val_model)

    weight_user = np.empty(len(part_of_model), dtype=float)
    for i in range(len(part_of_model)):
        for j in range(len(nama_user)):
            if part_of_model["nama"][i] == nama_user[j]:
                weight_user[i] = cos_sim_user[0][j]
        
    weight_content = np.empty(len(part_of_model), dtype=float)
    for i in range(len(part_of_model)):
        for j in range(len(nama_mua)):
            if part_of_model["nama_MUA"][i] == nama_mua[j]:
                weight_content[i] = value_mua[j]
        
    final_res = part_of_model.copy()
    final_res = final_res[['nama', 'nama_MUA']]
    final_res['user'] = weight_user
    final_res['content'] = weight_content

    mean_weight = np.zeros(len(part_of_model), dtype=float)
    for i in range(len(part_of_model)):
        mean_weight[i] = statistics.harmonic_mean([weight_user[i] + np.min(weight_user[np.nonzero(weight_user)]),  weight_content[i]]) 
    final_res['mean_'] = mean_weight
    higher_value = final_res.copy()
    higher_value = final_res.groupby("nama", as_index=False).mean_.agg(["mean"])
    u = final_res.groupby("nama", as_index=False).user.agg(["mean"])
    higher_value['weight_user'] = u['mean']
    c = final_res.groupby("nama", as_index=False).content.agg(["mean"])
    higher_value['weight_content'] = c['mean']
    higher_value = higher_value.sort_values(by=['mean'], ignore_index=True, ascending=False)
    print(higher_value)
    list_nama = higher_value.nama[:10].tolist()
    pivot = part_of_model.copy()
    pivot['mean_'] = mean_weight
    pivot = pivot[pivot.nama.isin(list_nama)].reset_index()
    distance = pivot[['nama_MUA','distance']]
    produk = pivot[['nama_MUA', 'sim']]
    final_ = pivot[['nama_MUA', 'sim_final']]
    desc = pivot[['nama_MUA', 'produk_makeup', 'shade', 'skin_color', 'skin_undertone']]
    desc['desc'] = desc['produk_makeup'] + ' - ' + desc['shade'] + ' - ' + desc['skin_color'] + ' - ' + desc['skin_undertone']
    pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
    pivot_['nama_MUA'] = desc.groupby('nama_MUA').first().index.to_list()
    pivot_['weight'] = pivot.groupby("nama_MUA").mean_.agg(['mean'])
    pivot_['score'] = pivot_["mean"] * pivot_['weight']
    pivot_['desc'] = desc.groupby('nama_MUA').agg({'desc': lambda x: list(x)}).reset_index()['desc'].values
    pivot_['distance'] = distance.groupby("nama_MUA").distance.agg(['mean'])
    pivot_['sim'] = produk.groupby("nama_MUA").sim.agg(['mean'])
    pivot_['sim_final'] = final_.groupby("nama_MUA").sim_final.agg(['mean'])
    pivot_ = pivot_.sort_values(by=['score'], ignore_index=True, ascending=False)
    return pivot_

In [27]:
#highest MAE
mae_eval[mae_eval.index(min(mae_eval))]

0.531153678312746

In [28]:
#highest
idx_high = mae_eval.index(min(mae_eval))
data_for_model.iloc[[selected_user[idx_high]]]

,no,nama,nama_MUA,latitude,longitude,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating,desc
73,80,dwi rahma,Bonita MUA Jember,-8.139019,113.731123,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,1,-8.139019,113.728548,4,lt pro smooth corrector cream foundation natur...


In [29]:
h_pivot = testing_user(data_for_model, data_for_user, idx_high)
h_pivot

                              nama      mean  weight_user  weight_content
0                        dwi rahma  1.111120     1.000000        1.037629
1                     zyanti laila  0.679624     0.312348        1.025218
2         vika aprilia sinta putri  0.565236     0.195959        1.013600
3              priska sisilia dewi  0.330111     0.000000        1.046609
4                 frensisca amelia  0.330111     0.000000        1.046609
5                    eka febrianti  0.329539     0.000000        1.035221
6                olivia ramadhania  0.329221     0.000000        1.028965
7                  diajeng agustin  0.329168     0.000000        1.027944
8            moh wahyu nur hidayah  0.329168     0.000000        1.027944
9                        adera zia  0.328983     0.000000        1.024441
10          zainata hanum masruroh  0.328976     0.000000        1.024368
11                       azwa sasa  0.328960     0.000000        1.023894
12                   anggun yeriam  0.

,count,mean,nama_MUA,weight,score,desc,distance,sim,sim_final
0,3,4.333333,Ayuk Rias Pengantin,0.814793,3.530768,[LT Pro smooth corrector cream foundation - na...,21.84,0.305435,0.013373
1,2,3.000000,Iqlim Rias Pengantin,1.116615,3.349846,[LT Pro smooth corrector cream foundation - na...,9.06,0.547320,0.054406
2,4,3.500000,Sanggar Rias Pengantin Dewi Balqis,0.842187,2.947654,[la tulipe cover foundation - rachel 2 (kuning...,3.90,0.305718,0.062391
3,2,5.000000,Sanggar Rias Cantik Tina,0.564158,2.820792,[la tulipe cover foundation - rachel 1 (kuning...,11.35,0.176355,0.014280
4,2,5.000000,Griya Putri Make Up,0.563997,2.819983,[la tulipe cover foundation - rachel 2 (kuning...,4.52,0.064116,0.011615
5,2,4.000000,Yasmin MUA dan Henna Jember,0.682241,2.728963,[la tulipe cover foundation - rachel 1 (kuning...,11.69,0.547320,0.043130
6,2,4.000000,Laura Make Up & Bridal,0.678769,2.715075,[la tulipe cover foundation - natural (putih) ...,7.62,0.143185,0.016611
7,4,4.500000,Larasnada Make Up,0.506205,2.277923,[la tulipe cover foundation - rachel 1 (kuning...,2.78,0.309097,0.081772
8,2,4.000000,Hiekmaa MUA,0.562965,2.251859,[la tulipe cover foundation - natural (putih) ...,8.39,0.064116,0.006828
9,4,4.000000,Dinda Wijaya Make Up,0.503866,2.015463,[Ultima II Wonderwear Cream Make Up - linen - ...,13.72,0.389092,0.026433


In [43]:
h_pivot.to_excel('highest.xlsx')

In [ ]:
#lowest MAE
mae_eval[mae_eval.index(max(mae_eval))]

0.8707099526302452

In [31]:
#lowest
idx_low = mae_eval.index(max(mae_eval))
data_for_model.iloc[[selected_user[idx_low]]]

,no,nama,nama_MUA,latitude,longitude,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating,desc
0,53,adera zia,Dinda Wijaya Make Up,-8.193711,113.619405,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,1,-8.193711,113.61683,5,lt pro smooth corrector cream foundation natur...


In [32]:
h_pivot = testing_user(data_for_model, data_for_user, idx_low)
h_pivot[h_pivot.score > 0.0]

                              nama      mean  weight_user  weight_content
0                        adera zia  1.086104     1.000000        1.000734
1                     zyanti laila  0.718459     0.365575        1.025961
2                        dwi rahma  0.666860     0.307579        1.021947
3           zainata hanum masruroh  0.643759     0.288534        0.994396
4                     maulina vira  0.567756     0.210851        0.988448
5                    anissa serina  0.564175     0.206958        0.990898
6        callista febriana erwanto  0.545166     0.187409        0.999472
7            moh wahyu nur hidayah  0.317524     0.000000        1.038632
8                  diajeng agustin  0.317524     0.000000        1.038632
9           azahra aura wiritanaya  0.317307     0.000000        1.034011
10           sonia arista firnanda  0.317307     0.000000        1.034011
11             priska sisilia dewi  0.316965     0.000000        1.026785
12                frensisca amelia  0.

,count,mean,nama_MUA,weight,score,desc,distance,sim,sim_final
0,5,4.200000,Larasnada Make Up,0.830397,3.487668,[la tulipe cover foundation - rachel 1 (kuning...,15.88,0.254114,0.015054
1,2,5.000000,Wedding Gallery Make Over,0.643074,3.215369,[la tulipe cover foundation - rachel 1 (kuning...,9.93,0.141086,0.012908
2,3,5.000000,Winda Wijaya MUA & Beauty Studio,0.617301,3.086506,[la tulipe cover foundation - natural (putih) ...,13.51,0.127911,0.008815
3,3,4.333333,Ayuk Rias Pengantin,0.698894,3.028540,[Wardah Everyday Luminous Liquid Foundation - ...,8.20,0.306357,0.033300
4,7,3.857143,Bonita MUA Jember,0.765883,2.954120,[la tulipe cover foundation - rachel 1 (kuning...,13.72,0.321512,0.021842
5,2,4.000000,Yasmin MUA dan Henna Jember,0.732087,2.928348,[LT Pro smooth corrector cream foundation - na...,3.29,0.547495,0.127621
6,3,4.333333,Laura Make Up & Bridal,0.665965,2.885847,[la tulipe cover foundation - natural (putih) ...,6.76,0.106394,0.013711
7,4,3.500000,Ardie Make Up & Salon,0.817142,2.859997,[la tulipe cover foundation - natural (putih) ...,6.70,0.187158,0.024306
8,3,5.000000,Rias Manten Mariyam,0.548173,2.740866,[LT Pro smooth corrector cream foundation - na...,8.44,0.377333,0.039972
9,2,5.000000,Salon Fatah,0.544011,2.720053,[LT Pro smooth corrector cream foundation - na...,27.06,0.473771,0.016884


In [33]:
h_pivot.to_excel('lowest.xlsx')

## COLLABORATIVE FILTERING

In [34]:
def colab_evaluation(data_for_model, data_for_user, selected_user, query_user):
    mae_col = []
    for u in range(len(selected_user)):
        _, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[u], query_user[u])
        nama_mua, _, part_of_model = value_of_mua(query_data_todict, part_of_model)
        pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        col_model = pivot.columns.tolist()
        val_model = np.array(pivot.values.tolist(), dtype=float)
        nama_user = np.array(pivot.index.tolist())
        pivot_query = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_query = pivot_query.values.tolist()
        col_query = pivot_query.columns.tolist()
        feature = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_query)):
                if col_model[i] == col_query[j]:
                    feature[i] = val_query[0][j]
                    break
        knn = NearestNeighbors(metric='cosine', algorithm='brute')
        knn.fit(val_model)
        distances, indices = knn.kneighbors([feature], n_neighbors=10)
        list_nama = []
        for ln in indices[0]:
             list_nama.append(nama_user[ln])
        pivot = data_for_model[data_for_model.nama.isin(list_nama)].reset_index()
        tmp_cos = np.empty(len(pivot), dtype=float)
        for i in range(len(pivot.nama)):
            for j in range(len(list_nama)):
                if pivot.nama[i] == list_nama[j]:
                    tmp_cos[i] = distances[0][j]
        pivot['cos'] = tmp_cos
        pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
        pivot_['cos'] = pivot.groupby("nama_MUA").cos.agg(['mean'])
        value_rating = np.zeros(len(col_model), dtype=float)
        for i in range(len(pivot_.index)):
            for j in range(len(col_model)):
                if value_rating[j] == 0.0:
                    if col_model[j] == pivot_.index[i]:
                        value_rating[j] = pivot_["mean"][i] * pivot_["cos"][i]
                else:
                    continue
        tmp_data_recs = {'nama_mua':nama_mua, 'score':value_rating}
        mua_recs = pd.DataFrame(tmp_data_recs).sort_values(by=['score'], ignore_index=True, ascending=False)
        pivot_label = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_label = pivot_label.values.tolist()
        col_label = pivot_label.columns.tolist()
        final_label = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_label)):
                if col_model[i] == col_label[j]:
                    final_label[i] = val_label[0][j]
                    break
        mae = mean_absolute_error(value_rating, final_label)
        mae_col.append(mae)
        
    mean_mae_col = np.mean(mae_col)
    return mua_recs, mean_mae_col

In [35]:
colab_based_fil, mae_col = colab_evaluation(data_for_model, data_for_user, selected_user, query_user)

In [36]:
colab_based_fil[colab_based_fil.score > 0.0]

,nama_mua,score
0,Gasebo Make Up Dekorasi,5.000000
1,Salon Fatah,5.000000
2,Rias Pengantin Sapphire,5.000000
3,Yayuk Makeover,5.000000
4,Rias Ning Lida,4.000000
5,Ayunia Bridal dan Salon,3.106061
6,Sanggar Rias Pengantin Dewi Balqis,2.787879
7,Eka Make Over,2.484848
8,Bonita MUA Jember,2.333333
9,Griya Putri Make Up,2.096955


In [37]:
mae_col

1.1881830605638575

## CONTENT BASED FILTERING

In [38]:
def content_evaluation(data_for_model, data_for_user, selected_user, query_user):
    mae_con = []
    for u in range(len(selected_user)):
        _, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[u], query_user[u])
        nama_mua, _, part_of_model = value_of_mua(query_data_todict, part_of_model)
        pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        col_model = pivot.columns.tolist()
        get_user = part_of_model.groupby("nama").sim_num.agg(["mean"])
        get_user = get_user.sort_values(by=["mean"], ascending=False)
        list_nama = get_user.index[:10].tolist()
        pivot = part_of_model[part_of_model.nama.isin(list_nama)].reset_index()
        pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
        pivot_['cos'] = pivot.groupby("nama_MUA").total_mua.agg(['mean'])
        value_rating = np.zeros(len(col_model), dtype=float)
        for i in range(len(pivot_.index)):
            for j in range(len(col_model)):
                if value_rating[j] == 0.0:
                    if col_model[j] == pivot_.index[i]:
                        value_rating[j] = pivot_["mean"][i] * pivot_["cos"][i]
                else:
                    continue
        tmp_data_recs = {'nama_mua':nama_mua, 'score':value_rating}
        mua_recs = pd.DataFrame(tmp_data_recs).sort_values(by=['score'], ignore_index=True, ascending=False)
        pivot_label = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_label = pivot_label.values.tolist()
        col_label = pivot_label.columns.tolist()
        final_label = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_label)):
                if col_model[i] == col_label[j]:
                    final_label[i] = val_label[0][j]
                    break
        mae = mean_absolute_error(value_rating, final_label)
        mae_con.append(mae)
        
    mean_mae_con = np.mean(mae_con)
    return mua_recs, mean_mae_con

In [39]:
content_based_fil, mae_con = content_evaluation(data_for_model, data_for_user, selected_user, query_user)

In [40]:
content_based_fil[content_based_fil.score > 0.0]

,nama_mua,score
0,Gasebo Make Up Dekorasi,4.982277
1,Rias Pengantin Sapphire,4.913576
2,Ifa Salon,4.035091
3,Rosa Indah Griya Rias Pengantin,3.966176
4,Rias Manten Mariyam,3.943948
5,Griya Citra Ayu,3.846099
6,Ella Surya Make Up,3.824399
7,Rias Pengantin Azizah,2.980570
8,Salon Vinell,2.924440


In [41]:
mae_con

0.9482621504176993

In [44]:
data_evaluasi = {
    'Metode' : ['Collaborative Filtering', 'Content Based Filtering', 'Hybrid Filtering'],
    'MAE' : [mae_col, mae_con, mae_hyb]
}

tabel_evaluasi = pd.DataFrame(data_evaluasi)
tabel_evaluasi = tabel_evaluasi.sort_values(by=['MAE'], ignore_index=True)
tabel_evaluasi

,Metode,MAE
0,Hybrid Filtering,0.735606
1,Content Based Filtering,0.948262
2,Collaborative Filtering,1.188183
